In [1]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  
import calendar
import argparse
import glob

In [2]:
run = 'parallel_new_input/'
os.chdir('/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_support_code/'+run)

print(os.getcwd()) 


/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_support_code/parallel_new_input


In [3]:
output_col_names = ['lon'
,'lat'
,'variable'
,'cohort_number'
,'stand_age'
,'potential_veg'
,'current_veg'
,'subtype'
,'community_type'
,'silt_clay'  ###check this order
,'lc_state'
,'land_area'
,'cohort_area'
,'year'
,'annual_sum'
,'monthly_maximum'
,'monthly_mean'
,'monthly_minimum'
,'Jan'
,'Feb'
,'Mar'
,'Apr'
,'May'
,'Jun'
,'Jul'
,'Aug'
,'Sep'
,'Oct'
,'Nov'
,'Dec'
,'region']

input_col_names = ["lon", 'lat','var' ,'Area', 'year', 'sum', 'max', 'average'
         , 'min', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'
         , 'Nov', 'Dec', 'Area_Name']

In [4]:
read_in_data = pd.read_csv('/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_support_code/'+run+'all_parallel_variable_info.csv')

In [5]:
read_in_data

,input_group,NIRR,PAR,PREC,TAIR,VPR,WS10,output_paths,output_group,output_var
0,0,clm_out/NIRR.csv0,clm_out/PAR.csv0,clm_out/PREC.csv0,clm_out/TAIR.csv0,clm_out/VPR.csv0,clm_out/WS10.csv0,var_out/SOILORGC.csv0,0,SOILORGC
1,0,clm_out/NIRR.csv0,clm_out/PAR.csv0,clm_out/PREC.csv0,clm_out/TAIR.csv0,clm_out/VPR.csv0,clm_out/WS10.csv0,var_out/NPP.csv0,0,NPP
2,0,clm_out/NIRR.csv0,clm_out/PAR.csv0,clm_out/PREC.csv0,clm_out/TAIR.csv0,clm_out/VPR.csv0,clm_out/WS10.csv0,var_out/VEGC.csv0,0,VEGC
3,0,clm_out/NIRR.csv0,clm_out/PAR.csv0,clm_out/PREC.csv0,clm_out/TAIR.csv0,clm_out/VPR.csv0,clm_out/WS10.csv0,var_out/GPP.csv0,0,GPP
4,10024,clm_out/NIRR.csv10024,clm_out/PAR.csv10024,clm_out/PREC.csv10024,clm_out/TAIR.csv10024,clm_out/VPR.csv10024,clm_out/WS10.csv10024,var_out/GPP.csv10024,10024,GPP
...,...,...,...,...,...,...,...,...,...,...
195,7518,clm_out/NIRR.csv7518,clm_out/PAR.csv7518,clm_out/PREC.csv7518,clm_out/TAIR.csv7518,clm_out/VPR.csv7518,clm_out/WS10.csv7518,var_out/NPP.csv7518,7518,NPP
196,8771,clm_out/NIRR.csv8771,clm_out/PAR.csv8771,clm_out/PREC.csv8771,clm_out/TAIR.csv8771,clm_out/VPR.csv8771,clm_out/WS10.csv8771,var_out/NPP.csv8771,8771,NPP
197,8771,clm_out/NIRR.csv8771,clm_out/PAR.csv8771,clm_out/PREC.csv8771,clm_out/TAIR.csv8771,clm_out/VPR.csv8771,clm_out/WS10.csv8771,var_out/VEGC.csv8771,8771,VEGC
198,8771,clm_out/NIRR.csv8771,clm_out/PAR.csv8771,clm_out/PREC.csv8771,clm_out/TAIR.csv8771,clm_out/VPR.csv8771,clm_out/WS10.csv8771,var_out/SOILORGC.csv8771,8771,SOILORGC


In [6]:
i = 4

In [7]:
##read in precipitation files
prec = pd.read_csv(read_in_data.loc[i, 'PREC']
           , names = input_col_names)
par = pd.read_csv(read_in_data.loc[i, 'PAR']
           , names = input_col_names)
nirr = pd.read_csv(read_in_data.loc[i, 'NIRR']
           , names = input_col_names)
tair = pd.read_csv(read_in_data.loc[i, 'TAIR']
           , names = input_col_names)
vpr = pd.read_csv(read_in_data.loc[i, 'VPR']
           , names = input_col_names)
ws10 = pd.read_csv(read_in_data.loc[i, 'WS10']
           , names = input_col_names)


print('data read in')
## shape input datasets wide to long
prec = pd.melt(prec, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='prec'
        ,col_level=0)

par = pd.melt(par, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='par'
        ,col_level=0)

nirr = pd.melt(nirr, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='nirr'
        ,col_level=0)

tair = pd.melt(tair, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='tair'
        ,col_level=0)

vpr = pd.melt(vpr, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='vpr'
        ,col_level=0)

ws10 = pd.melt(ws10, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='ws10'
        ,col_level=0)

##merge climate variables together
climate_vars = prec.merge(
            par,on = ['lon','lat', 'Area', 'year', 'month']
            ).merge(
            tair ,on = ['lon','lat', 'Area', 'year', 'month']                           
            ).merge(
            nirr ,on = ['lon','lat', 'Area', 'year', 'month']                           
            ).merge(
            vpr ,on = ['lon','lat', 'Area', 'year', 'month']                           
            ).merge(
            ws10 ,on = ['lon','lat', 'Area', 'year', 'month']                           
            )



data read in


In [8]:
del prec, par, tair, nirr, vpr, ws10
##subset to forest veg types 
# cohort_output = cohort_output.loc[cohort_output['current_veg'].isin(forest_vegs)] 
print('clim processed')


clim processed


In [9]:
climate_vars
print('climate rows : '+ str(len(climate_vars)))

climate rows : 136800


In [10]:
###read in cohort output
cohort_output = pd.read_csv(read_in_data.loc[i, 'output_paths']
            , names = output_col_names)

print('cohort rows read in : '+ str(len(cohort_output)))

cohort rows read in : 460159


In [78]:
# cohort_output

In [11]:
###summary table of lat, lon, num years, num cohorts, 
summary_lon_lat = cohort_output.groupby(['lon', 'lat'])[['year', 'cohort_number']].nunique()
summary_lon_lat.to_csv('/home/smmrrr/TEM_output_processed/first_checks_parallel/lon_lat_counts/'+read_in_data.loc[i, 'output_var']+str(read_in_data.loc[i, 'output_group'])+'.csv'
                        ,index = False)

In [12]:
##melt cohort output
cohort_output = pd.melt(cohort_output, id_vars = ['lon','lat', 'cohort_area', 'land_area', 'year','variable', 'current_veg', 'stand_age'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='value'
        ,col_level=0)

print('cohort rows wide to long : '+ str(len(cohort_output)))

cohort rows wide to long : 5521908


In [13]:
cohort_output

,lon,lat,cohort_area,land_area,year,variable,current_veg,stand_age,month,value
0,-85.0,67.5,1170,1170,1501,GPP,3,3000,Jan,0.00
1,-85.0,67.5,1170,1170,1502,GPP,3,3000,Jan,0.00
2,-85.0,67.5,1170,1170,1503,GPP,3,3000,Jan,0.00
3,-85.0,67.5,1170,1170,1504,GPP,3,3000,Jan,0.00
4,-85.0,67.5,1170,1170,1505,GPP,3,3000,Jan,0.00
...,...,...,...,...,...,...,...,...,...,...
5521903,-84.5,10.0,3,3041,2022,GPP,16,1,Dec,241.94
5521904,-84.5,10.0,1,3041,2022,GPP,48,1,Dec,50.83
5521905,-84.5,10.0,1,3041,2022,GPP,50,0,Dec,2.98
5521906,-84.5,10.0,2,3041,2022,GPP,16,0,Dec,137.73


In [14]:
#### match pfts to the current veg, bin by standage for forests
conditions = [
 cohort_output['current_veg'].isin([50, 49, 52, 53, 54, 55, 56]) 
    ,cohort_output['current_veg'].isin([51, 47])
, cohort_output['current_veg'].isin([48, 46]) 
    ,cohort_output['current_veg'].isin([4, 5, 6, 8, 9, 
                                        10, 11, 16, 17, 18, 19, 20, 25, 33])
    ,cohort_output['current_veg'].isin([7, 12, 13, 14, 22, 23, 24, 25, 26, 27, 28, 30, 31])
    ,cohort_output['current_veg'].isin([15, 35, 29])
    ,cohort_output['current_veg'].isin([2, 3])
    ,cohort_output['current_veg'].isin([21])
]

values = [
          'crop', 
          'pasture', 
          'urban or suburban', 
          'forest', 
          'grass', 
          'shrub', 
          'tundra',
            'desert']
cohort_output['pft'] = np.select(conditions, values, default = 'other')

intervals_standage = [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 3000]
cohort_output['stand_age_bin'] = pd.cut(
    cohort_output['stand_age'], bins=intervals_standage)
# print(annual_current_veg[['pft', 'stand_age_bin']].value_counts())
cohort_output.loc[cohort_output['pft']!='forest', ['stand_age_bin']] =  cohort_output['stand_age_bin'].min()
cohort_output['stand_age_interval_min'] = cohort_output['stand_age_bin'].apply(lambda x: x.right).astype(int)

In [104]:
# cohort_output.loc[cohort_output['stand_age_bin'].isna()]

,lon,lat,cohort_area,land_area,year,variable,current_veg,stand_age,month,value,pft,stand_age_bin


In [15]:
cohort_output['temp_weight'] = cohort_output['value'] * cohort_output['cohort_area'] 


aggregations = {
 'cohort_area':'sum',
 'land_area':'sum',
 'value':'mean',
    'temp_weight':'sum'
}


cohort_output = cohort_output.groupby(
    ['lon','lat','variable','pft','current_veg','year','month' ,'stand_age_interval_min']
).agg(
aggregations
)


# all_grids =  all_grids.reset_index()

cohort_output['value_weight'] = cohort_output['temp_weight']/cohort_output['cohort_area']
cohort_output = cohort_output.reset_index()
print('cohort rows post cohort collapse : '+ str(len(cohort_output)))
# cohort_output

cohort rows post cohort collapse : 268944


In [16]:
forest_vegs = [4, 5, 6, 8, 9, 10, 11, 16, 17, 18, 19, 20, 25, 33]
forest_types = ["Boreal Forest", "Forested Boreal Wetlands", "Boreal Woodlands","Mixed Temperate Forests", 
               "Temperate Coniferous Forests", "Temperate Deciduous Forests", "Temperate Forested Wetlands", 
               "Tropical Evergreen Forests", "Tropical Forested Wetlands", "Tropical Deciduous Forests", "Xeromorphic Forests and Woodlands"
               ,"Tropical Forested Floodplains", "Temperate Forested Floodplains", "Temperate Broadleaved Evergreen Forests"]

forest_pfts = pd.DataFrame({
'current_veg':forest_vegs, 
    'forest_type':forest_types
})

cohort_output=cohort_output.merge(forest_pfts, on = 'current_veg', how = 'left')

merged_dataset = cohort_output.merge(
            climate_vars, how = 'left'
    ,on = ['lon','lat', 'year', 'month']
            )

In [17]:
merged_dataset.columns

Index(['lon', 'lat', 'variable', 'pft', 'current_veg', 'year', 'month',
       'stand_age_interval_min', 'cohort_area', 'land_area', 'value',
       'temp_weight', 'value_weight', 'forest_type', 'Area', 'prec', 'par',
       'tair', 'nirr', 'vpr', 'ws10'],
      dtype='object')

In [90]:
# lon_lat_pfts

merged_dataset.to_csv('/home/smmrrr/TEM_output_processed/first_checks_parallel/lon_lat_pfts/'+read_in_data.loc[i, 'output_var']+str(read_in_data.loc[i, 'output_group'])+'.csv'
                        ,index = False)

merged_dataset.loc[merged_dataset['pft']=='forest'].to_csv('/home/smmrrr/TEM_output_processed/first_checks_parallel/forest_lon_lat_pfts/'+read_in_data.loc[i, 'output_var']+str(read_in_data.loc[i, 'output_group'])+'.csv'
                        ,index = False)


